In [1]:
import kaggle
import pandas as pd
import zipfile
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose

#biblioteca para fazer fazer a regressão
from sklearn.tree import DecisionTreeRegressor

#biblioteca para calcular uma métrica de avaliação
from sklearn.metrics import mean_squared_error

In [2]:
#kaggle.api.authenticate() #Autenticação tem que ocorrer usando o .json de acesso com suas credenciais do Kaggle
#criar link com os metadados das pesquisas e locais dos dados

In [3]:
#!kaggle datasets download -d andrevandor/hdi-1990-2018-20200529

In [4]:
#data_zip = zipfile.ZipFile('.\hdi-1990-2018-20200529.zip')
#data_zip.extractall('.\data')
#data_zip.close()

In [5]:
df = pd.read_excel('.\data\HDI_1990_2018.xlsx')

In [6]:
df.shape

(5481, 6)

In [7]:
df.head()

,Paises,Ano,Expectativa de vida no nascimento,Anos esperados de escolaridade,Média de anos de escolaridade,Renda nacional bruta (RNB) per capita (PPP US$)
0,Norway,1990,76.729,14.0,10.47,17910.0
1,Switzerland,1990,77.568,13.4,10.19,28170.0
2,Ireland,1990,74.809,12.1,9.73,12750.0
3,Germany,1990,75.488,14.4,8.77,19640.0
4,China,1990,69.145,8.8,5.34,17760.0


In [8]:
def EDA (df):
    eda_df = {}
    eda_df['Qtde_NaN'] = df.isnull().sum()
    eda_df['%_NaN'] = df.isnull().mean().round(2)
    eda_df['DType'] = df.dtypes
    eda_df['Qtde_Unicos'] = df.count()
    
    colunas = sorted(df.columns.tolist(), key=str.lower, reverse=False)
    eda_df['Qtde_Unicos'] = filtered_result = list(map(lambda x: len(df[x].unique().tolist()), colunas))
    
    eda_df['Media'] = df.mean().round(3)
    eda_df['Min'] = df.min()
    eda_df['Max'] = df.max()
    eda_df['STD'] = df.std().round(3)
    
    return pd.DataFrame(eda_df)

In [9]:
EDA(df.drop(['Ano','Paises'], axis = 1))

,Qtde_NaN,%_NaN,DType,Qtde_Unicos,Media,Min,Max,STD
Expectativa de vida no nascimento,84,0.02,float64,372,68.237,26.172,84.68700,9.566
Anos esperados de escolaridade,279,0.05,float64,4911,11.787,2.100,23.30000,3.348
Média de anos de escolaridade,551,0.10,float64,431,7.469,0.290,14.13215,3.163
Renda nacional bruta (RNB) per capita (PPP US$),450,0.08,float64,2507,13297.971,270.000,131510.00000,16413.508


In [10]:
df_1 = df.copy()
df_1['Expectativa de vida no nascimento'].fillna(df_1['Expectativa de vida no nascimento'].mean(), inplace = True)
df_1['Anos esperados de escolaridade'].fillna(df_1['Anos esperados de escolaridade'].mean(), inplace = True)
df_1['Média de anos de escolaridade'].fillna(df_1['Média de anos de escolaridade'].mean(), inplace = True)
df_1['Renda nacional bruta (RNB) per capita (PPP US$)'].\
    fillna(df_1['Renda nacional bruta (RNB) per capita (PPP US$)'].mode()[0], inplace = True)

EDA(df_1.drop(['Ano','Paises'], axis = 1))

,Qtde_NaN,%_NaN,DType,Qtde_Unicos,Media,Min,Max,STD
Expectativa de vida no nascimento,0,0.0,float64,372,68.237,26.172,84.68700,9.492
Anos esperados de escolaridade,0,0.0,float64,4911,11.787,2.100,23.30000,3.262
Média de anos de escolaridade,0,0.0,float64,431,7.469,0.290,14.13215,3.000
Renda nacional bruta (RNB) per capita (PPP US$),0,0.0,float64,2506,12311.274,270.000,131510.00000,16067.583


In [11]:
paises = df_1.Paises.unique()

In [12]:
index_paises = []
for i in range(len(paises)):
    index_paises.append (df_1[df_1.Paises == paises[i]].index)

In [13]:
df_2 = pd.DataFrame()
for i in range(len(index_paises)):
    df_2 = pd.concat([df_2, df_1.iloc[index_paises[i]]], axis = 0)

In [14]:
df_2.reset_index(inplace=True)
df_2.drop('index', axis=1, inplace=True)

In [15]:
index_paises = []
for i in range(len(paises)):
    index_paises.append (df_2[df_2.Paises == paises[i]].index)

In [17]:
def feature_engineering (df, horizonte, paises, index_paises):
    
    data = df.copy()
    
    horizonte += 1

    for i in range(len(paises)):
        #
        data.loc[index_paises[i],'Diff_1'] = data.iloc[index_paises[i]]['Expectativa de vida no nascimento']\
                                            .diff(horizonte - 1)
        data.loc[index_paises[i],'Diff_2'] = data.iloc[index_paises[i]]['Anos esperados de escolaridade']\
                                            .diff(horizonte - 1)
        data.loc[index_paises[i],'Diff_3'] = data.iloc[index_paises[i]]['Média de anos de escolaridade']\
                                            .diff(horizonte - 1)
        data.loc[index_paises[i],'Diff_4'] = data.iloc[index_paises[i]]['Renda nacional bruta (RNB) per capita (PPP US$)']\
                                            .diff(horizonte - 1)
        #
        data.loc[index_paises[i], 'Med_1'] = data.iloc[index_paises[i]]['Expectativa de vida no nascimento']\
                                            .rolling(horizonte).mean().round(2)
        data.loc[index_paises[i], 'Med_2'] = data.iloc[index_paises[i]]['Anos esperados de escolaridade']\
                                            .rolling(horizonte).mean().round(2)
        data.loc[index_paises[i], 'Med_3'] = data.iloc[index_paises[i]]['Média de anos de escolaridade']\
                                            .rolling(horizonte).mean().round(2)
        data.loc[index_paises[i], 'Med_4'] = data.iloc[index_paises[i]]['Renda nacional bruta (RNB) per capita (PPP US$)']\
                                            .rolling(horizonte).mean().round(2)
        #
        data.loc[index_paises[i], 'Var_%_1'] = data.iloc[index_paises[i]]['Expectativa de vida no nascimento']\
                                            .pct_change(horizonte - 1).round(horizonte)
        data.loc[index_paises[i], 'Var_%_2'] = data.iloc[index_paises[i]]['Anos esperados de escolaridade']\
                                            .pct_change(horizonte - 1).round(horizonte)
        data.loc[index_paises[i], 'Var_%_3'] = data.iloc[index_paises[i]]['Média de anos de escolaridade']\
                                            .pct_change(horizonte - 1).round(horizonte)
        data.loc[index_paises[i], 'Var_%_4'] = data.iloc[index_paises[i]]['Renda nacional bruta (RNB) per capita (PPP US$)']\
                                            .pct_change(horizonte - 1).round(horizonte)
    
    data.dropna(how='any', axis=0, inplace=True)
    
    return data

In [16]:
def split_prev(df, ano_ref, ano_prev, pais):

    data = df.copy()  
    
    treino =  data.loc[(data.Ano <= ano_ref) & (data.Paises == pais)].copy()
    teste = data.loc[(data.Ano > ano_ref) & (data.Ano <= ano_prev) & (data.Paises == pais)].copy()

    #não iremos utilizar essa coluna para treinar nosso modelo
    treino.drop(['Ano', 'Paises'], axis=1, inplace=True)
    teste.drop(['Ano', 'Paises'], axis=1, inplace=True)

    #vamos dividir as variáveis alvo e de entrada
    y_treino = treino['Expectativa de vida no nascimento'].values
    del treino['Expectativa de vida no nascimento']
    
    x_treino = treino.values

    y_teste = teste['Expectativa de vida no nascimento'].values
    del teste['Expectativa de vida no nascimento']

    x_teste = teste.values;

    #treino do modelo para fazer a previsão
    regr_1 = DecisionTreeRegressor(max_depth=5, min_samples_split = 2)
    regr_1.fit(x_treino, y_treino)

    y_pred_teste = regr_1.predict(x_teste)
    y_pred_treino = regr_1.predict(x_treino)

    # Avaliar o fitting
    rmse_teste = np.round(mean_squared_error(y_teste, y_pred_teste) **0.5, 2)
    rmse_treino = np.round(mean_squared_error(y_treino, y_pred_treino)**0.5, 2)
    
    return ano_ref, ano_prev, pais, rmse_teste, rmse_treino

In [18]:
def walk_forward (data, janela, horizonte, passo, limite):
    result = []
    pais_ref = data.Paises.unique()
    controle = limite - janela

    while controle >= 0:
        ano_ref = anos[:len(anos) - controle].max()
        ano_prev = ano_ref + horizonte
                
        for i in paises:
             result.append(split_prev(data, ano_ref, ano_prev, i))

        controle -= passo
    return result

In [41]:
def analise_estabilidade(data, janela, limite, paises, index_paises):
    result = []
    controle = limite - janela
    
    ano_ref = janela
    ano_prev = janela
    
    while controle >= 0:
        ano_prev += 1
    
        for i in paises:
             result.append(split_prev(data, ano_ref, ano_prev, i))

        controle -= 1
    
    return result

In [25]:
anos = df_2.Ano.head(df_2.Ano.max() - df_2.Ano.min()).values #nosso ultimo slice da base df principal
janela = 2005 #fará o walk de janela + horizonte, nesse caso pegará do menor ano até 2005 conjunto de treino.
horizonte = 1 #quantos anos para frente tentará prever
passo = 1 #no loop soma esse valor a janela.
limite = anos.max() #quando o loop para.

In [26]:
df_3 = feature_engineering (df_2, horizonte, paises, index_paises)

In [27]:
df_walk_forward = pd.DataFrame(data = walk_forward (df_3, janela, horizonte, passo, limite),\
                          columns=['Ano_Ref','Ano_Prev', 'Pais','Rmse_teste','Rmse_treino'])

In [31]:
df_walk_forward.head()

,Ano_Ref,Ano_Prev,Pais,Rmse_teste,Rmse_treino
0,2005,2006,Norway,0.50,0.0
1,2005,2006,Switzerland,0.25,0.0
2,2005,2006,Ireland,0.35,0.0
3,2005,2006,Germany,0.21,0.0
4,2005,2006,China,0.29,0.0


In [42]:
df_estabilidade = pd.DataFrame(data =  analise_estabilidade(df_3, janela, limite, paises, index_paises),\
                               columns=['Ano_Ref','Ano_Prev', 'Pais','Rmse_teste','Rmse_treino'])

In [48]:
df_estabilidade.tail()

,Ano_Ref,Ano_Prev,Pais,Rmse_teste,Rmse_treino
2426,2005,2018,Burundi,5.33,0.01
2427,2005,2018,South Sudan,4.11,0.00
2428,2005,2018,Chad,3.87,0.00
2429,2005,2018,Central African Republic,2.08,0.00
2430,2005,2018,Niger,5.67,0.00
